# BON CODE

In [41]:
import ipycanvas
import ipywidgets as widgets
import os
import chess
import chess.pgn
from stockfish import Stockfish
import math

# Configuration de l'échiquier
taille_case = 50
echiquier_size = 8
chemin_images = "chess_images"
canvas = ipycanvas.Canvas(width=taille_case * echiquier_size, height=taille_case * echiquier_size)
display(canvas)

stockfish = Stockfish("/usr/games/stockfish")  
stockfish.set_skill_level(10)

echiquier = chess.Board()
coups = []
indice_coup = 0
historique_positions = [echiquier.fen()]  

annotation_image_widget = widgets.Output()  # Widget pour afficher l'image d'annotation
display(annotation_image_widget)


def charger_partie_pgn(fichier_pgn):
    with open(fichier_pgn, "r") as pgn:
        game = chess.pgn.read_game(pgn)
    return list(game.mainline_moves())


def dessiner_echiquier():
    for y in range(echiquier_size):
        for x in range(echiquier_size):
            color = "white" if (x + y) % 2 == 0 else "grey"
            canvas.fill_style = color
            canvas.fill_rect(x * taille_case, y * taille_case, taille_case, taille_case)


def dessiner_piece_image(image_path, x, y):
    if os.path.exists(image_path):
        with open(image_path, "rb") as file:
            image = widgets.Image(value=file.read(), format='png')
        canvas.draw_image(image, x * taille_case, y * taille_case, taille_case, taille_case)


def mettre_a_jour_echiquier():
    canvas.clear()
    dessiner_echiquier()
    for case in chess.SQUARES:
        piece = echiquier.piece_at(case)
        if piece:
            x = chess.square_file(case)
            y = 7 - chess.square_rank(case)
            image_path = os.path.join(chemin_images, f"{'w' if piece.color else 'b'}{piece.symbol().upper()}.png")
            dessiner_piece_image(image_path, x, y)


def evaluer_coup(move, coup_index):
    """Analyse un coup avec Stockfish et retourne une annotation et le chemin de son image."""
    
    temp_board = chess.Board()
    
    for i in range(coup_index):
        temp_board.push(coups[i])
    
    if move not in temp_board.legal_moves:
        return "Coup illégal", None  # Pas d'image pour un coup illégal
   
    stockfish.set_fen_position(temp_board.fen())
    eval_avant = stockfish.get_evaluation().get('value', 0)
    temp_board.push(move)
    stockfish.set_fen_position(temp_board.fen())
    eval_apres = stockfish.get_evaluation().get('value', 0)
    variation = eval_apres - eval_avant

    # Associer les annotations aux images avec les critères ajustés
    if variation >= 300:
        return "Brillant", "annotations_images/brilliant.png"  # Brilliant Move
    elif 150 <= variation < 300:
        return "Très bon coup", "annotations_images/tres bon.png"  # Very Good Move
    elif 50 <= variation < 150:
        return "Meilleur coup", "annotations_images/meilleur coup.png"  # Best Move
    elif 0 <= variation < 50:
        return "!", "annotations_images/bon.png"  # Good Move
    elif -50 <= variation < 0:
        return "?!?", "annotations_images/imprecision.png"  # Inaccuracy
    elif -100 <= variation < -50:
        return "?", "annotations_images/interessant.png"  # Mistake
    elif variation < -100:
        return "??", "annotations_images/gaffe.png"  # Blunder
    else:
        return "=",

    
def dessiner_annotation_sur_case(x, y, image_path):
    """Dessine une petite image d'annotation en haut à droite d'une case."""
    if os.path.exists(image_path):
        with open(image_path, "rb") as file:
            image = widgets.Image(value=file.read(), format='png')

        # Taille réduite de l'image (ex: 20x20 pixels)
        canvas.draw_image(image, x * taille_case + 30, y * taille_case, 20, 20)


def afficher_annotation_image(image_path):
    """Affiche une image d'annotation en tant que widget en dessous de l'échiquier."""
    annotation_image_widget.clear_output()
    
    if image_path and os.path.exists(image_path):
        with annotation_image_widget:
            with open(image_path, "rb") as file:
                image = widgets.Image(value=file.read(), format='png')
            display(image)


def afficher_eval_actuelle(b):
    """Affiche l'annotation sur la case et dessine une flèche vers le meilleur coup alternatif."""
    if indice_coup > 0:
        move = coups[indice_coup - 1]
        annotation, image_path = evaluer_coup(move, indice_coup - 1)

        # Créer un échiquier temporaire et revenir au coup précédent
        temp_board = chess.Board()
        for i in range(indice_coup - 1):
            temp_board.push(coups[i])

        # Demander à Stockfish le meilleur coup à ce moment-là
        stockfish.set_fen_position(temp_board.fen())
        meilleur_coup = stockfish.get_best_move()
        
        if meilleur_coup:
            meilleur_move = chess.Move.from_uci(meilleur_coup)

            # Extraire les coordonnées
            x1, y1 = chess.square_file(meilleur_move.from_square), 7 - chess.square_rank(meilleur_move.from_square)
            x2, y2 = chess.square_file(meilleur_move.to_square), 7 - chess.square_rank(meilleur_move.to_square)

            # Dessiner la flèche verte pour montrer le meilleur coup que le joueur aurait dû jouer
            dessiner_fleche(x1, y1, x2, y2, couleur="green")

        # Afficher l'image de l'évaluation sur la case du coup joué
        if image_path and os.path.exists(image_path):
            case = move.to_square
            x = chess.square_file(case)
            y = 7 - chess.square_rank(case)
            dessiner_annotation_sur_case(x, y, image_path)



def dessiner_fleche(x1, y1, x2, y2, couleur="green"):
    """Dessine une flèche entre deux cases."""
    canvas.stroke_style = couleur
    canvas.line_width = 3
    canvas.begin_path()
    canvas.move_to(x1 * taille_case + taille_case // 2, y1 * taille_case + taille_case // 2)
    canvas.line_to(x2 * taille_case + taille_case // 2, y2 * taille_case + taille_case // 2)
    canvas.stroke()

    # Dessiner la pointe de la flèche
    angle = math.atan2((y2 - y1), (x2 - x1))
    arrow_size = 10
    for i in [-1, 1]:  # Deux segments pour former la pointe
        end_x = x2 * taille_case + taille_case // 2 - arrow_size * math.cos(angle + i * math.pi / 6)
        end_y = y2 * taille_case + taille_case // 2 - arrow_size * math.sin(angle + i * math.pi / 6)
        canvas.begin_path()
        canvas.move_to(x2 * taille_case + taille_case // 2, y2 * taille_case + taille_case // 2)
        canvas.line_to(end_x, end_y)
        canvas.stroke()



def rejouer_partie(fichier_pgn):
    global echiquier, coups, indice_coup, historique_positions
    echiquier = chess.Board()
    coups = charger_partie_pgn(fichier_pgn)
    indice_coup = 0
    historique_positions = [echiquier.fen()]
    mettre_a_jour_echiquier()

    bouton_suivant = widgets.Button(description="Coup suivant")
    bouton_suivant.on_click(jouer_prochain_coup)
    bouton_precedent = widgets.Button(description="Coup précédent")
    bouton_precedent.on_click(jouer_ancien_coup)
    bouton_evaluation = widgets.Button(description="Afficher évaluation")
    bouton_evaluation.on_click(afficher_eval_actuelle)

    display(bouton_suivant, bouton_precedent, bouton_evaluation)


def jouer_prochain_coup(b):
    global echiquier, indice_coup
    if indice_coup < len(coups):
        echiquier.push(coups[indice_coup])
        historique_positions.append(echiquier.fen())
        indice_coup += 1
        mettre_a_jour_echiquier()


def jouer_ancien_coup(b):
    global echiquier, indice_coup
    if indice_coup > 0:
        echiquier = chess.Board(historique_positions[indice_coup - 1])
        indice_coup -= 1
        mettre_a_jour_echiquier()


# Charger la partie
rejouer_partie("Loowolf_vs_TelophileGOAT_2025.02.19.pgn")


Canvas(height=400, width=400)

Output()

Button(description='Coup suivant', style=ButtonStyle())

Button(description='Coup précédent', style=ButtonStyle())

Button(description='Afficher évaluation', style=ButtonStyle())